In [ ]:
import pandas as pd
data=pd.read_csv('https://690v.blob.core.windows.net/690v/breakdown_with_wind.csv',encoding='latin-1')

In [136]:
import numpy as np
from bokeh.models import ColumnDataSource,Range1d,Plot,Rect
from bokeh.models.glyphs import ImageURL
from bokeh.io import output_notebook, show
import math

mapurl="https://user-images.githubusercontent.com/31782589/32149092-1f761dd0-bcd6-11e7-9077-44459a42a7ef.png"
arrowurl="https://user-images.githubusercontent.com/31782589/32187993-ec43dcb6-bd7c-11e7-9ca8-6eae18e2b6f2.png"

sizefactor=1.6
imagewidth=521 #ori img size yue = 5210*2650
imageheight=265
actualwidth=833#int(sizefactor*imagewidth)
actualheight=424#int(sizefactor*imageheight)
dotsize=int(sizefactor*4)

#get initial data on 4/30
initialdata430=data[(data.created_month==4) & (data.created_day==30)]

imagesource = ColumnDataSource(dict(url = [mapurl]))
windsource=ColumnDataSource(dict(url = [arrowurl],size=[initialdata430['windsize'].iloc[0]],angle=[initialdata430['wind_angle'].iloc[0]]))

plot = Plot(x_range=Range1d(0,actualwidth), y_range=Range1d(0,actualheight), plot_width=actualwidth, plot_height=actualheight,min_border=0)
plot.title.text="Twitter posts on 4/30 cloudy"
plot.title.align='center'

mapimg = ImageURL(url="url", x=0, y=0, w=actualwidth, h=actualheight, anchor="bottom_left")
windimg=ImageURL(url="url", x=833/2, y=100,w="size",h="size",anchor="bottom_left",angle="angle",global_alpha=0.4)

plot.add_glyph(imagesource, mapimg)
windhandle=plot.add_glyph(windsource, windimg)


datasource = ColumnDataSource(data=initialdata430.head())
twittersquares = Rect(x="defaultx", y="defaulty", width=dotsize, height=dotsize, angle=0, fill_color="red",line_color="red")
twitterhandle=plot.add_glyph(datasource, twittersquares)

output_notebook()
show(plot,notebook_handle=True)

Loading BokehJS ...

In [137]:
from bokeh.io import push_notebook
import ipywidgets as widgets
from IPython.display import display
import datetime

nextdaybutton = widgets.Button(description="5/1")
previousdaybutton = widgets.Button(description="4/29")
currentdate=datetime.date(2011,4,30)
display(previousdaybutton)
display(nextdaybutton)

#data part

def nextdaygo(button):
    global currentdate
    previousdate=currentdate
    currentdate=currentdate+datetime.timedelta(days=1)
    nextdate=currentdate+datetime.timedelta(days=1)
    previousdaybutton.description=str(previousdate.month)+'/'+str(previousdate.day)
    nextdaybutton.description=str(nextdate.month)+'/'+str(nextdate.day)
    updatemapdata(currentdate)
    
def previousdaygo(button):
    global currentdate
    nextdate=currentdate
    currentdate=currentdate-datetime.timedelta(days=1)
    previousdate=currentdate-datetime.timedelta(days=1)
    previousdaybutton.description=str(previousdate.month)+'/'+str(previousdate.day)
    nextdaybutton.description=str(nextdate.month)+'/'+str(nextdate.day)
    updatemapdata(currentdate)
    
def updatemapdata(date):
    #date is the date to extract data to show on map
    currentdatedata=data[(data.created_month==date.month) & (data.created_day==date.day)]
    twitterhandle.data_source.data=dict(defaultx=currentdatedata.head()['defaultx'],defaulty=currentdatedata.head()['defaulty'])
    windhandle.data_source.data=dict(url = [arrowurl],size=[currentdatedata['windsize'].iloc[0]],angle=[currentdatedata['wind_angle'].iloc[0]])
    plot.title.text="Twitter posts on "+ str(date.month)+'/'+str(date.day)+' '+currentdatedata.Weather.iloc[0]
    push_notebook()
    
nextdaybutton.on_click(nextdaygo)
previousdaybutton.on_click(previousdaygo)